<a href="https://colab.research.google.com/github/dp1500/News-Aggregator/blob/main/News_Aggregator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score

In [ ]:
def read_articles_from_json_files(json_files):
    """
    Reads articles from multiple JSON files and consolidates them into a single list.
    """
    all_articles = []
    for file_path in json_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            all_articles.extend(data)  # Extend the list with articles directly from JSON
    return all_articles

# Path to your JSON files
json_files = ['/content/ndtv (1).json', '/content/times_of_india (1).json', '/content/zee_news (1).json']

# Read articles from JSON files
news_data = read_articles_from_json_files(json_files)

# Convert news_data to DataFrame
df = pd.DataFrame(news_data)

In [ ]:
df.head()

,id,title,sub_heading,date_and_auther,artical_link,time,content,source,article_link,subtitle
0,R3DLZJNTYTE8C3B,"PM Modi Unveils Projects Worth Rs 12,800 Crore...",Prime Minister Narendra Modi on Wednesday unve...,"Press Trust of India | Wednesday March 06, 2...",https://www.ndtv.com/india-news/pm-modi-unveil...,,[PM Modi laid the foundation stone and dedicat...,ndtv,NaN,NaN
1,ZM2CJBSCTFC8HW6,"""Let's Unite"": Mamata Banerjee Calls For Massi...","On Wednesday, the same day when Prime Minister...",Asian News International | Wednesday March 0...,https://www.ndtv.com/india-news/lets-unite-mam...,,"[""Bengal is cultural capital of India and it s...",ndtv,NaN,NaN
2,O3JEJ1BXR8TDAUC,"Man Slaps Elderly Mother, Hits Her With Slippe...","Slipper in hand, he hits the elderly woman rep...",\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tEdited by Ashu...,https://www.ndtv.com/india-news/man-slaps-elde...,,[A video grab shows the man thrashing his moth...,ndtv,NaN,NaN
3,FBX3J0MIHGSGOWL,"Another Gujarat Congress MLA Resigns, 4th In 3...",Gujarat Congress MLA Arvind Ladani on Wednesda...,"Press Trust of India | Wednesday March 06, 2...",https://www.ndtv.com/india-news/another-gujara...,,"[After submitting his resignation, Arvind Lada...",ndtv,NaN,NaN
4,GMNIWA4S199GHSU,Former Trinamool Leader Tapas Roy Joins BJP,"Former TMC leader Tapas Roy, who had quit West...","Press Trust of India | Wednesday March 06, 2...",https://www.ndtv.com/india-news/former-trinamo...,,"[Tapas Roy, a five-time MLA, had accused the T...",ndtv,NaN,NaN


In [ ]:
df.tail()

,id,title,sub_heading,date_and_auther,artical_link,time,content,source,article_link,subtitle
279,MRQ159Z781KMO2H,Shillong Teer Result TODAY 06.03.2024 First An...,Shillong Teer Result Wednesday 06.03.2024: On ...,NaN,https://zeenews.india.com/india/shillong-teer-...,,Shillong Teer Result 2024: The Shillong Teer L...,zee_news,NaN,NaN
280,2KM2RUMARUELG55,Congress To Promise 'Right To Employment' To Y...,The measures come after Rahul Gandhi during th...,NaN,https://zeenews.india.com/india/congress-to-pr...,,NEW DELHI: Seeking to woo the country's youth ...,zee_news,NaN,NaN
281,UO7X02P14MS9MN5,"In Mamata Banerjee's Fortress Bengal, PM Modi ...",Prime Minister Narendra Modi called the women ...,NaN,https://zeenews.india.com/india/in-mamata-bane...,,New Delhi: Prime Minister Narendra Modi on Wed...,zee_news,NaN,NaN
282,MI3A5XJ9R4DV1Q7,"From Bengal's Barasat, PM Modi Warns TMC Of 'S...",PM Modi's emphasis on women's empowerment came...,NaN,https://zeenews.india.com/india/from-bengals-b...,,Prime Minister Narendra Modi today blasted the...,zee_news,NaN,NaN
283,218Y59GHF4417HA,Peru PM Alberto Otarola Resigns Over Influence...,The issue was raised after the Panorama TV bro...,NaN,https://zeenews.india.com/india/peru-pm-albert...,,Peru's powerful Prime Minister Alberto Otarola...,zee_news,NaN,NaN


In [ ]:
df['sub_heading'].isna().sum()

54

In [ ]:
df['content'].isna().sum()

13

In [ ]:
df['content'].isna()

0      False
1      False
2      False
3      False
4      False
       ...  
279    False
280    False
281    False
282    False
283    False
Name: content, Length: 284, dtype: bool

In [ ]:
df_test = df[df['content'].isna()]

In [ ]:
df_test

In [ ]:
df.shape

(284, 10)

In [ ]:
# Drop rows where 'content' column is NaN
df.dropna(subset=['content'], inplace=True)
df.shape

(271, 10)

# Pre Processing

In [ ]:
df['title'] = df['title'].apply(lambda x : x.lower())
df.iloc[0].title

'pm modi unveils projects worth rs 12,800 crore in bihar'

### removing punctuation

In [ ]:
import string

def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    return s

df['title'] = df['title'].apply(remove_punctuation)
df.iloc[0].title

'pm modi unveils projects worth rs 12800 crore in bihar'

### Stemming

In [ ]:
ps = PorterStemmer()
def stem(text):
    x = []

    for i in text.split():
        x.append(ps.stem(i))
    return " ".join(x)

In [ ]:
df['title'] = df['title'].apply(stem)
df.iloc[0].title

'fierc gunfight near senior cop hous in manipur imphal armi call in'

#CountVectorizing

In [ ]:
cv = CountVectorizer(max_features=5000,stop_words='english')

* here max_features can be changed to manage how the model performs

In [ ]:
vectors = cv.fit_transform(df['title']).toarray()


In [ ]:
vectors.shape

(271, 1215)

In [ ]:
# print(cv.get_feature_names_out())
feature_names = cv.get_feature_names_out()
for feature in feature_names:
    print(feature)


0409pm
06032024
10
100
1000hour
108
12
12800
13
14yearold
16
19600
1st
20
2007
2019
2022
2024
2024bjp
2024delhi
2024panauti
2024rahul
2024rameshwaram
2024sandeshkhali
2024tiger
21
28
2nd
30
30year
370
3year
45
472
4th
500
520
6yearjail
7200
75
89yearold
8yearold
90
99
aap
aaps
abhijit
abroad
abrogation
accepts
accord
accused
accuseds
accuses
acquittal
acquitted
action
activities
actor
actress
adani
address
adityanath
advises
advisory
agencies
agency
agencys
agra
ahead
ai
aiadmk
aide
air6
aircraft
airport
ajit
akali
alberto
allahabad
alleged
alliance
allies
allocation
aloud
amethi
amid
amit
amrit
amritsar
andhra
animal
announced
announces
annual
antiterror
anurag
apartment
apollo
apologises
app
approaches
approval
apps
areas
argument
arjun
army
arrested
article
arunachal
arvind
asha
asked
asks
assam
assault
assaulted
assaulting
assembly
assures
astronauts
atm
attaches
attack
attacked
attacks
attempting
authorities
authority
avalanches
avoid
award
awarded
ayodhya
backing
badals
bail
balb

# Simillarity

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [ ]:
similarity

array([[1.        , 0.        , 0.        , ..., 0.22222222, 0.21081851,
        0.12598816],
       [0.        , 1.        , 0.        , ..., 0.10540926, 0.1       ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.22222222, 0.10540926, 0.        , ..., 1.        , 0.31622777,
        0.12598816],
       [0.21081851, 0.1       , 0.        , ..., 0.31622777, 1.        ,
        0.11952286],
       [0.12598816, 0.        , 0.        , ..., 0.12598816, 0.11952286,
        1.        ]])

In [ ]:
similarity.shape

(271, 271)

* so similarity is a matrix where each row is article and columns are also articles, the intersection element provides the similarity value

# Processing similar articles

In [ ]:
# def recommend(movie):
#     index = new[new['title'] == movie].index[0]    # fetches the index of movie in dataset
#     distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
#     for i in distances[1:6]:
#         print(new.iloc[i[0]].title)

In [ ]:
import numpy as np

# Assume similarity_matrix is your 285x285 similarity matrix
# Iterate over the first 10 articles
for i in range(24):
    # Get the similarity scores for the current article i
    similarity_scores = similarity[i]
    # Sort the similarity scores and get the indices of the top 5 most similar articles
    top_5_indices = np.argsort(similarity_scores)[::-1][:5]
    # Print the information of the current article and its top 5 similar articles
    print(f"Article {i}: {df.iloc[i]['title']}")
    print("Top 5 most similar articles:")
    for index in top_5_indices:
        if index != i:  # Exclude self-similarity
            print(f"- {df.iloc[index]['title']} (Similarity Score: {similarity_scores[index]})")
    print()


Article 0: pm modi unveils projects worth rs 12800 crore in bihar
Top 5 most similar articles:
- pm modi unveils projects worth rs 12800 crore in bihar (Similarity Score: 0.9999999999999999)
- pm modi to unveil projects worth rs 12800 crore in bihar (Similarity Score: 0.8888888888888888)
- pm modi launches development projects worth over rs 19600 crore in odisha (Similarity Score: 0.6324555320336759)
- pm modi launches projects worth rs 7200 crore in telangana highlights centres focus (Similarity Score: 0.5773502691896258)

Article 1: lets unite mamata banerjee calls for massive rally in kolkata on march 10
Top 5 most similar articles:
- lets unite mamata banerjee calls for massive rally in kolkata on march 10 (Similarity Score: 0.9999999999999999)
- after massive turnout at pm modis rally women from sandeshkhali meet him (Similarity Score: 0.22360679774997896)
- bank holidays in march 2024 (Similarity Score: 0.15811388300841897)
- 0409pm modi inaugurates indias first underwater metro 

# Clustering

In [ ]:
def cluster_articles(similarity_matrix, threshold):
    num_articles = len(similarity_matrix)
    clusters = {}
    assigned = set()  # Keep track of articles already assigned to a cluster

    for i in range(num_articles):
        if i not in assigned:  # If article i is not assigned to any cluster
            cluster = [i]  # Initialize a new cluster with article i
            assigned.add(i)  # Mark article i as assigned

            for j in range(i + 1, num_articles):
                if j not in assigned and similarity_matrix[i, j] >= threshold:
                    cluster.append(j)  # Add similar article j to the cluster
                    assigned.add(j)  # Mark article j as assigned

            clusters[len(clusters) + 1] = cluster  # Add the cluster to the clusters dictionary

    return clusters

# Assuming similarity_matrix is your 285x285 similarity matrix and threshold is 0.35
threshold = 0.32
clusters = cluster_articles(similarity, threshold)

# Print the total number of clusters formed
print(f"Total number of clusters: {len(clusters)}")

# Print all the clusters
for cluster_num, articles in clusters.items():
    print(f"Cluster {cluster_num}:")
    for article_idx in articles:
        article_info = df.iloc[article_idx]
        print(f"{article_info['source']}: {article_info['title']} ||", end=" ")
    print()  # Add a new line after each cluster


Total number of clusters: 172
Cluster 1:
ndtv: pm modi unveils projects worth rs 12800 crore in bihar || ndtv: pm modi unveils projects worth rs 12800 crore in bihar || ndtv: pm modi to unveil projects worth rs 12800 crore in bihar || ndtv: pm modi launches development projects worth over rs 19600 crore in odisha || ndtv: pm modi launches projects worth rs 7200 crore in telangana highlights centres focus || 
Cluster 2:
ndtv: lets unite mamata banerjee calls for massive rally in kolkata on march 10 || ndtv: lets unite mamata banerjee calls for massive rally in kolkata on march 10 || 
Cluster 3:
ndtv: man slaps elderly mother hits her with slipper in up || ndtv: man slaps elderly mother hits her with slipper in up || 
Cluster 4:
ndtv: another gujarat congress mla resigns 4th in 3 months || ndtv: another gujarat congress mla resigns 4th in 3 months || 
Cluster 5:
ndtv: former trinamool leader tapas roy joins bjp || ndtv: former trinamool leader tapas roy joins bjp || times_of_india: forme

In [ ]:
# # Save the output to a text file
# output_file_path = "clusters_output.txt"
# with open(output_file_path, "w") as f:
#     # Write the total number of clusters
#     f.write(f"Total number of clusters: {len(clusters)}\n\n")

#     # Write all the clusters
#     for cluster_num, articles in clusters.items():
#         f.write(f"Cluster {cluster_num}:\n")
#         for article_idx in articles:
#             article_info = df.iloc[article_idx]
#             f.write(f"{article_info['source']}: {article_info['title']} , {article_info['id']} || ")
#         f.write("\n\n")

# print("Output saved to:", output_file_path)

Output saved to: clusters_output.txt


# Keyword extraction

### TF-IDF approach (bad)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Define a function to preprocess text
def clean_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    return text

# Define a stemmer object
stemmer = PorterStemmer()

# Function to apply stemming to tokens
def stemming_tokenizer(text):
    return [stemmer.stem(token) for token in text.split()]

# Function to extract keywords from text using TF-IDF
def extract_keywords(text):
    # Preprocess text
    cleaned_text = clean_text(text)

    # Apply TF-IDF vectorization
    tfidf_matrix = tfidf_vectorizer.fit_transform([cleaned_text])

    # Get feature names (words) from TF-IDF vectorizer
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Extract keywords with highest TF-IDF scores
    tfidf_scores = zip(feature_names, tfidf_matrix.toarray()[0])
    sorted_tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)
    top_keywords = [word for word, _ in sorted_tfidf_scores[:5]]  # Extract top 5 keywords
    return top_keywords

# Define a function to extract keywords from news clusters
def extract_keywords_from_clusters(clusters, df):
    cluster_keywords = {}

    for cluster_num, articles_idx in clusters.items():
        # Combine titles and content of articles in the cluster
        cluster_texts = []
        for article_idx in articles_idx:
            title = df.loc[article_idx, 'title']
            content = df.loc[article_idx, 'content']
            combined_text = f"{title} {content}"
            cluster_texts.append(combined_text)

        # Extract keywords for each article in the cluster
        cluster_keywords[cluster_num] = []
        for text in cluster_texts:
            keywords = extract_keywords(text)
            cluster_keywords[cluster_num].append(keywords)

    return cluster_keywords

# Assuming 'clusters' is a dictionary where keys are cluster numbers and values are lists of article indices
# Assuming 'df' is a DataFrame containing news articles with columns 'title' and 'content'

# Extract keywords from news clusters
keywords_by_cluster = extract_keywords_from_clusters(clusters, df)

# Print keywords for each cluster
for cluster_num, keywords in keywords_by_cluster.items():
    print(f"Cluster {cluster_num} Keywords:")
    for article_keywords in keywords:
        print(", ".join(article_keywords))
    print()


### Page Rank Approach

In [ ]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
# nltk.download('wordnet ')
# nltk.download('wordnet ')
# nltk.download('wordnet ')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import networkx as nx
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Function to preprocess text
def preprocess_text(text):
    words = word_tokenize(text)  # Tokenize text
    words = [word for word in words if word.isalpha() and word.lower() not in stopwords.words('english')]  # Remove stopwords and non-alphabetic tokens
    tagged_words = pos_tag(words)  # Perform Part-of-Speech tagging
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word, pos=get_wordnet_pos(tag)) for word, tag in tagged_words]  # Lemmatize words
    return lemmatized_words

# Function to map Part-of-Speech tags to WordNet tags
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return 'a'  # Adjective
    elif treebank_tag.startswith('V'):
        return 'v'  # Verb
    elif treebank_tag.startswith('N'):
        return 'n'  # Noun
    elif treebank_tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun if POS tag is not recognized


# Function to extract keywords from text
def extract_keywords(text, num_keywords=5):
    cleaned_text = preprocess_text(text)  # Preprocess text
    G = nx.Graph()  # Create a graph
    G.add_nodes_from(cleaned_text)  # Add nodes (words) to the graph
    for i, word in enumerate(cleaned_text):  # Create edges between words based on co-occurrence
        for j in range(i + 1, min(i + 5, len(cleaned_text))):
            if not G.has_edge(word, cleaned_text[j]):
                G.add_edge(word, cleaned_text[j])
    scores = nx.pagerank(G)  # Calculate PageRank scores
    sorted_keywords = sorted(scores, key=scores.get, reverse=True)  # Sort keywords by PageRank scores
    return sorted_keywords[:num_keywords]  # Return top keywords

# Function to extract keywords from a cluster of articles
def extract_keywords_cluster(cluster_articles, num_keywords=6):
    all_keywords = []  # Initialize list to store keywords from all articles in the cluster
    for article_text in cluster_articles:  # Loop through each article in the cluster
        article_keywords = extract_keywords(article_text, num_keywords=num_keywords)  # Extract keywords from the current article
        all_keywords.extend(article_keywords)  # Add keywords from the current article to the list
    keyword_freq = {}  # Count the frequency of each keyword
    for keyword in all_keywords:
        if keyword in keyword_freq:
            keyword_freq[keyword] += 1
        else:
            keyword_freq[keyword] = 1
    sorted_keywords = sorted(keyword_freq, key=keyword_freq.get, reverse=True)  # Sort keywords by frequency
    return sorted_keywords[:num_keywords]  # Return top keywords

# Example usage:
# Assuming you have clusters dictionary containing clusters of articles
# clusters = {1: [article_idx1, article_idx2, ...], 2: [article_idx3, article_idx4, ...], ...}
for cluster_num, articles in clusters.items():  # Loop through each cluster
    print(f"Cluster {cluster_num}:")
    cluster_texts = [df.iloc[article_idx]['title'] for article_idx in articles]  # Extract titles of articles in the cluster
    cluster_keywords = extract_keywords_cluster(cluster_texts, num_keywords=5)  # Extract keywords from the cluster
    print("Keywords:", cluster_keywords)  # Print keywords
    for article_idx in articles:  # Print titles of articles in the cluster
        article_info = df.iloc[article_idx]
        print(f"{article_info['source']}: {article_info['title']} ||", end=" ")
    print()  # Add a new line after each cluster


Cluster 1:
Keywords: ['cop', 'house', 'senior', 'manipurs', 'near']
ndtv: fierce gunfight near senior cops house in manipurs imphal army called in || ndtv: fierce gunfight near senior cops house in manipurs imphal army called in || 
Cluster 2:
Keywords: ['walas', 'parent', 'expect', 'second', 'moose']
ndtv: sidhu moose walas friend composer escapes unhurt in restaurant firing || ndtv: sidhu moose walas parents expecting their second child in march || ndtv: sidhu moose walas parents expecting their second child in march || 
Cluster 3:
Keywords: ['inspire', 'fail', 'officer', 'get', 'ip']
ndtv: greatest happiness ips officer who inspired 12th fail gets tribute from his village school || ndtv: greatest happiness ips officer who inspired 12th fail gets tribute from his village school || 
Cluster 4:
Keywords: ['offer', 'passenger', 'free', 'taxi', 'rob']
ndtv: up man drugs robs passengers after offering free taxi rides arrested || ndtv: up man drugs robs passengers after offering free taxi 

In [ ]:
# Save the output to a text file
output_file_path = "keywords_output_2.txt"
with open(output_file_path, "w") as f:
    # Write keywords extracted from each cluster to the file
    for cluster_num, articles in clusters.items():
        f.write(f"Cluster {cluster_num}:\n")
        cluster_texts = [df.iloc[article_idx]['title'] for article_idx in articles]  # Extract titles of articles in the cluster
        cluster_keywords = extract_keywords_cluster(cluster_texts, num_keywords=6)  # Extract keywords from the cluster
        f.write("Keywords: " + ", ".join(cluster_keywords) + "\n")  # Write keywords to the file
        for article_idx in articles:  # Write titles of articles in the cluster
            article_info = df.iloc[article_idx]
            f.write(f"{article_info['source']}: {article_info['title']}\n")
        f.write("\n")  # Add a new line after each cluster

print("Output saved to:", output_file_path)


Output saved to: keywords_output_2.txt


# Scrapping twitter

In [ ]:
import tweepy
from datetime import datetime, timedelta

# Twitter API credentials
consumer_key = "d81Q9fmiQTd3aOsloVHwweZDL"
consumer_secret = "4baLjpzubdpQplBmwoYbCnUBhUC31B8wn0MbPfKoY29iyds6GA"
access_token = "1764176479754080256-jidJFD8yTQGKZPwGAS8spVxXd3SzX8%3Divhfa3gxoN5OuKhzpOUG4hvVSzsUCflyC3OfJNkoUwzACOiRvC"
access_token_secret = "fyhFDB4wKTyz9x0SGYXEskFDkPgSPzilkyB5psbrNpvB2"

# Authenticate to Twitter API
auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret, access_token, access_token_secret)
api = tweepy.API(auth)

# Define function to extract tweets based on keywords
def extract_tweets(keywords, num_tweets=50, until_date=None):
    tweets = []
    for keyword in keywords:
        if until_date:
            until_date_str = until_date.strftime('%Y-%m-%d')
            query = f"{keyword} until:{until_date_str}"
        else:
            query = keyword
        # Search for tweets
        tweet_objects = tweepy.Cursor(api.search_tweets, q=query, lang="en").items(num_tweets)
        for tweet in tweet_objects:
            tweets.append(tweet)
    return tweets

# Get current date
current_date = datetime.now().date()

# Define function to extract tweets for first 5 clusters
def extract_tweets_for_clusters(clusters, num_tweets=50, until_date=None):
    for cluster_num in range(1, min(6, len(clusters) + 1)):
        print(f"Cluster {cluster_num}:")
        articles = clusters[cluster_num]
        cluster_texts = [df.iloc[article_idx]['title'] for article_idx in articles]  # Extract titles of articles in the cluster
        cluster_keywords = extract_keywords_cluster(cluster_texts, num_keywords=5)  # Extract keywords from the cluster
        tweets = extract_tweets(cluster_keywords, num_tweets=num_tweets, until_date=until_date)  # Extract tweets based on keywords

        print("Tweets related to the cluster:")
        for tweet in tweets:
            print("Tweet Caption:", tweet.text)
            print("Date:", tweet.created_at)
            print("User:", tweet.user.screen_name)
            print()
        print("\n")  # Add a new line after each cluster

# Call the function to extract tweets for first 5 clusters
extract_tweets_for_clusters(clusters, num_tweets=50, until_date=current_date - timedelta(days=7))


Cluster 1:


BadRequest: 400 Bad Request
215 - Bad Authentication data.